In [99]:
import sys
sys.path.append('../../src')
from order_book import Book
#from order_book import Format
from event import Event
from datetime import datetime, time
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# set up logger to track events
import log
logger = log.get_logger('Main')
logger.info('testing logger from module Main')

# import lobster message file
cols = ['time', 'type', 'id', 'shares', 'price', 'direction']
data = pd.read_csv("../../data/lobster/MSFT_2012-06-21_34200000_37800000_message_50.csv", names=cols)
# re-scale the price col
data.price = data.price/10000
# make sure data is during market hours
data = data[data['time']>= 9.5*60*60]
data = data[data['time']<= 16*60*60]

print(len(data))
data.head()

141507


,time,type,id,shares,price,direction
0,34200.013994,3,16085616,100,31.04,-1
1,34200.013994,1,16116348,100,31.05,-1
2,34200.015248,1,16116658,100,31.04,-1
3,34200.015442,1,16116704,100,31.05,-1
4,34200.015789,1,16116752,100,31.06,-1


In [144]:
# read the messages and transform into an oderbook
book = Book()

for i in range(10_000):
    event = Event(data.loc[i])
    book.handleEvent(event, i)


In [145]:
# format the orderbook 
output = book.formatBook(start_from=100, levels=1)
output.head()

,Ask_1,Ask_1_Vol,Ask_1_Ord,Bid_1,Bid_1_Vol,Bid_1_Ord,Time
0,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197858
1,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197889
2,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197889
3,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197919
4,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197919


In [146]:
# add micro price
# S = (Pa * Vb / Va + Vb) + (Pb * Va / Va + Vb)
output['Micro_Price'] = ((output['Ask_1'] * output['Bid_1_Vol']) / (output['Bid_1_Vol'] + output['Ask_1_Vol'])) + ((output['Bid_1'] * output['Ask_1_Vol']) / (output['Bid_1_Vol'] + output['Ask_1_Vol']))
output.head()

,Ask_1,Ask_1_Vol,Ask_1_Ord,Bid_1,Bid_1_Vol,Bid_1_Ord,Time,Micro_Price
0,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197858,30.952143
1,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197889,30.952143
2,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197889,30.952143
3,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197919,30.952143
4,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197919,30.952143


In [147]:
# we can look at the orders in either all 5 levels or just the top level
bid_q, ask_q = book.getQueues(start_from=100, level=1)
bid_q.head()

,Bid_1,Time,l1
100,"[16135101.0, 30.95, 300.0]",03:30:00.197858,1
101,"[16135101.0, 30.95, 300.0]",03:30:00.197889,1
102,"[16135101.0, 30.95, 300.0]",03:30:00.197889,1
103,"[16135101.0, 30.95, 300.0]",03:30:00.197919,1
104,"[16135101.0, 30.95, 300.0]",03:30:00.197919,1


In [148]:
# to look at the lifecycle of an order, we'll start by getting all the order IDs we've stored 
all_ids = list(book.orders.keys())
order_metrics = []
# itterate through each id
for id in all_ids:
    # get the life cycle of each ID - this an order object attribute that is just a list that is updated 
    # each time an event takes place that affects an order that we've already read into the book
    
    order = book.orders.get(id)
    order_life_cycle = order.life
    submission_price = order.price
    side = order.direction
    starting_shares = order_life_cycle[0][1]
    types = []
    prices = []
    # within each life cycle save the event type and price (given the format we save is [time, shares, type, Price], we know the indicess are 2 and 3)
    # also check if the prices of events differer - we will be able to measure execution quality
    for x in order_life_cycle:
        event_type = x[2]
        price = x[3]
        types.append(event_type)
        prices.append(price)
    # save the unique event types for each ID
    unique_types = list(set(types))
    unique_prices = list(set(prices))
    # save the number of unique event types
    num_types = len(unique_types)
    num_prices = len(unique_prices)
    # mark orders that are fully executed
    if types[-1] == 4.0:
        full_ex = True
    else:
        full_ex = False
    # mark orders that have any execution
    if 4.0 in types:
        any_ex = True
    else:
        any_ex = False
    # add metrics to list
    order_metrics.append([id, len(order_life_cycle), unique_types, num_types, unique_prices, num_prices, starting_shares, submission_price, side, full_ex, any_ex])
# create df
order_metrics = pd.DataFrame(order_metrics, columns=['ID', 'Life', 'Unique_Event_Types', 'Num_Types', 'Unique_Prices', 'Num_Prices', 'Shares', 'Submission_Price', 'Side', 'Full_Ex', 'Any_Ex'])
# lets sort based on life - i.e. find the ID that has the most events that change it
order_metrics = order_metrics.sort_values('Life', ascending=False)
order_metrics.head()

,ID,Life,Unique_Event_Types,Num_Types,Unique_Prices,Num_Prices,Shares,Submission_Price,Side,Full_Ex,Any_Ex
304,1500599,10,"[1.0, 4.0]",2,[31.09],1,5556.0,31.09,-1.0,True,True
516,12642581,9,"[1.0, 4.0]",2,[31.11],1,1500.0,31.11,-1.0,True,True
1426,17287513,9,"[1.0, 4.0]",2,[31.0],1,3400.0,31.00,1.0,True,True
822,16818483,8,"[1.0, 4.0]",2,[30.97],1,2200.0,30.97,1.0,True,True
635,16716814,8,"[1.0, 4.0]",2,[31.02],1,2200.0,31.02,1.0,True,True


In [149]:
# check if the order was submitted at NBBO
at_bbo = []
for x in range(len(order_metrics)):
    id = order_metrics.iloc[x]['ID']
    order = book.orders.get(id)
    # get entry time and find book state at that time
    order_entry_time = datetime.fromtimestamp(order.entryTime).time()
    book_at_entry = output[output['Time']==order_entry_time]
    # given we trim the first 100 values there may be some empty values so handle these
    if book_at_entry.empty:
        at_bbo.append(None)
    # check side     
    elif order.direction == 1.0:
        if book_at_entry.iloc[0]['Bid_1'] == order.price:
            at_bbo.append(True)
        else:
            at_bbo.append(False)
    else:
        if book_at_entry.iloc[0]['Ask_1'] == order.price:
            at_bbo.append(True)
        else:
            at_bbo.append(False)

order_metrics["Sub_At_BBO"] = at_bbo
order_metrics.head()


,ID,Life,Unique_Event_Types,Num_Types,Unique_Prices,Num_Prices,Shares,Submission_Price,Side,Full_Ex,Any_Ex,Sub_At_BBO
304,1500599,10,"[1.0, 4.0]",2,[31.09],1,5556.0,31.09,-1.0,True,True,False
516,12642581,9,"[1.0, 4.0]",2,[31.11],1,1500.0,31.11,-1.0,True,True,False
1426,17287513,9,"[1.0, 4.0]",2,[31.0],1,3400.0,31.00,1.0,True,True,True
822,16818483,8,"[1.0, 4.0]",2,[30.97],1,2200.0,30.97,1.0,True,True,True
635,16716814,8,"[1.0, 4.0]",2,[31.02],1,2200.0,31.02,1.0,True,True,True


In [154]:
# above we see an order that goes through 8 seperate events so we will graph that order
# It is a large submission followed by a series of executions then a deletion
# MSFT order: 17287513
order = book.orders.get(17287513)
order_life = order.getLife()
order_life

,Time,Shares,Type,Price,Share_Life
0,03:30:18.105278,3400.0,1.0,31.0,3400.0
1,03:30:19.353072,500.0,4.0,31.0,2900.0
2,03:30:21.276276,800.0,4.0,31.0,2100.0
3,03:30:25.274548,200.0,4.0,31.0,1900.0
4,03:31:03.816079,143.0,4.0,31.0,1757.0
5,03:31:04.339811,200.0,4.0,31.0,1557.0
6,03:31:04.984306,300.0,4.0,31.0,1257.0
7,03:31:10.962947,100.0,4.0,31.0,1157.0
8,03:31:15.046305,1157.0,4.0,31.0,0.0


In [155]:
# Split out the types to graph
order_life_sub = order_life[order_life['Type'] == 1.0]
order_life_can = order_life[order_life['Type'] == 2.0]
order_life_del = order_life[order_life['Type'] == 3.0]
order_life_ex = order_life[order_life['Type'] == 4.0]

In [157]:
bid_trace = go.Scatter(x=output.Time, y=output.Bid_1, mode='lines',
                       line=dict(color='green'),
                       name='Bid Price')
ask_trace = go.Scatter(x=output.Time, y=output.Ask_1, mode='lines',
                       line=dict(color='red'),
                       name='Ask Price')
micro_trace = go.Scatter(x=output.Time, y=output.Micro_Price, mode='lines',
                       line=dict(color='black'),
                       name='Micro Price')
bid_vol_trace = go.Bar(x=output.Time, y=output.Bid_1_Vol,
                       marker=dict(color='green'),
                       name='Bid Volume')
ask_vol_trace = go.Bar(x=output.Time, y=output.Ask_1_Vol,
                       marker=dict(color='red'),
                       name='Ask Volume')
ask_q_trace = go.Scatter(x=ask_q.Time, y=ask_q.l1, mode='lines',
                         line=dict(color='red'),
                         name='Best Ask Q Len')
bid_q_trace = go.Scatter(x=bid_q.Time, y=bid_q.l1, mode='lines',
                         line=dict(color='green'),
                         name='Best Bid Q Len')
order_vol_trace = go.Bar(x=order_life.Time, y=order_life.Share_Life,
                       marker=dict(color='blue'),
                       name='Order Volume Remaining')
order_trace_sub = go.Scatter(x=order_life_sub.Time, y=order_life_sub.Price, mode='markers',
                         marker=dict(color='blue', size=np.log(order_life_sub.Shares)*5),
                         name='Order submission',
                         text='Price: '+order_life_sub.Price.astype(str)+'<br>'+'Shares: '+order_life_sub.Shares.astype(str))
order_trace_can = go.Scatter(x=order_life_can.Time, y=order_life_can.Price, mode='markers',
                         marker=dict(color='orange', size=np.log(order_life_can.Shares)*5),
                         name='Order cancelation',
                         text='Price: '+order_life_can.Price.astype(str)+'<br>'+'Shares: '+order_life_can.Shares.astype(str))
order_trace_del = go.Scatter(x=order_life_del.Time, y=order_life_del.Price, mode='markers',
                         marker=dict(color='black', size=np.log(order_life_del.Shares)*5),
                         name='Order deletion',
                         text='Price: '+order_life_del.Price.astype(str)+'<br>'+'Shares: '+order_life_del.Shares.astype(str))
order_trace_ex = go.Scatter(x=order_life_ex.Time, y=order_life_ex.Price, mode='markers',
                         marker=dict(color='purple', size=np.log(order_life_ex.Shares)*5),
                         name='Order execution',
                         text='Price: '+order_life_ex.Price.astype(str)+'<br>'+'Shares: '+order_life_ex.Shares.astype(str))

lens = [bid_trace, 
        ask_trace, 
        bid_vol_trace, 
        ask_vol_trace, 
        ask_q_trace,
        bid_q_trace, 
        order_vol_trace, 
        order_trace_sub,
        order_trace_can, 
        order_trace_del, 
        order_trace_ex,
        micro_trace]

fig = go.Figure(data=lens).set_subplots(4,1, subplot_titles=['Top of Book Price and Single Order Life Cycle', 'Top of Book Volume', 'Best Ask / Bid Q Length', 'Single Order Shares Remaining'],vertical_spacing=0.05, row_heights=[0.4, 0.2, 0.2, 0.2], shared_xaxes=True)
fig.data[2].xaxis = "x2"
fig.data[2].yaxis = "y2"
fig.data[3].xaxis = "x2"
fig.data[3].yaxis = "y2"
fig.data[4].xaxis = "x3"
fig.data[4].yaxis = "y3"
fig.data[5].xaxis = "x3"
fig.data[5].yaxis = "y3"
fig.data[6].xaxis = "x4"
fig.data[6].yaxis = "y4"

fig.update_layout(yaxis_title='Price')
fig.update_layout(yaxis2_title='Shares', yaxis2_range=[0,500_000])
fig.update_layout(yaxis3_title='Queue Length')
fig.update_layout(yaxis4_title='Shares', xaxis4_title='Time')
fig.update_layout(width=1440, height=1200, margin=dict(l=50, r=50, b=75, t=50))
fig.update_layout(barmode='stack')
#fig.show()

Look at order 17287513

!['Order Life'](../data/images/single_order.png)

Looking closer, we can see the Queue lenght tick up when the order is submitted as well as the volume at the limit.
Following this, the queue lenght drops and we see the first execution

!['Order Life 2'](../data/images/single_order_2.png)

In [119]:
# get time based data 
full_ex_times = []
first_ex_times = []
ex_duration_times = []
start_positions = []
nums_ex = []
vols_ex = []
avgs_ex = []

# itterate though the orders
for x in range(len(order_metrics)):
    # get order life
    id = order_metrics.iloc[x]['ID']
    order = book.orders.get(id)
    order_life = order.getLife()
    # save the submission in queue position
    start_positions.append(order.start_position)

    # get each submission time and the final event time
    submission_time = datetime.combine(datetime.today(), order_life.iloc[0]['Time'])
    # check if full executuin has take place
    if order_metrics.iloc[x]['Any_Ex'] == True:
        # filter to execution events
        ex_data = order_life[order_life['Type'] == 4.0]
        # get num executions and total shares ex
        num_ex = len(ex_data)
        vol_ex = sum(ex_data['Shares'])
        # get avg ex size
        avg_ex = vol_ex / num_ex

        nums_ex.append(num_ex)
        vols_ex.append(vol_ex)
        avgs_ex.append(avg_ex)
        # get first and last ex size
        first_ex = datetime.combine(datetime.today(),ex_data.iloc[0]['Time'])
        final_ex = datetime.combine(datetime.today(), ex_data.iloc[-1]['Time'])

        # calculate time to first and full execute
        time_to_first_ex = first_ex-submission_time
        time_to_final_ex = final_ex-submission_time
        ex_duration = final_ex - first_ex

        full_ex_times.append(time_to_final_ex.total_seconds())
        first_ex_times.append(time_to_first_ex.total_seconds())
        ex_duration_times.append(ex_duration.total_seconds())
    else:
        nums_ex.append(0)
        vols_ex.append(0)
        avgs_ex.append(0)
        full_ex_times.append(0)
        first_ex_times.append(0)
        ex_duration_times.append(0)

# add to df and add a share bucket column
order_metrics['Num_Executions'] = nums_ex
order_metrics['Vol_Executed'] = vols_ex
order_metrics['Avg_Ex_Size'] = avgs_ex
order_metrics['Full_Ex_Time'] = full_ex_times
order_metrics['First_Ex_Time'] = first_ex_times
order_metrics['Ex_Duration'] = ex_duration_times
order_metrics['Start_Position'] = start_positions
order_metrics['Share_Bucket'] = ((order_metrics['Shares'] // 100) * 100) + 1

order_metrics.head()

,ID,Life,Unique_Event_Types,Num_Types,Unique_Prices,Num_Prices,Shares,Submission_Price,Side,Full_Ex,Any_Ex,Sub_At_BBO,Num_Executions,Vol_Executed,Avg_Ex_Size,Full_Ex_Time,First_Ex_Time,Ex_Duration,Start_Position,Share_Bucket
7376,22789996,13,"[1.0, 2.0, 3.0]",3,[30.96],1,100.0,30.96,1.0,False,False,True,0,0.0,0.000000,0.000000,0.000000,0.000000,9,101.0
7512,22791405,13,"[1.0, 4.0]",2,[30.97],1,100.0,30.97,1.0,True,True,True,12,100.0,8.333333,21.308662,4.831894,16.476768,8,101.0
1426,17287513,9,"[1.0, 4.0]",2,[31.0],1,3400.0,31.00,1.0,True,True,True,8,3400.0,425.000000,56.941027,1.247794,55.693233,3,3401.0
20811,31773092,9,"[1.0, 4.0]",2,[31.01],1,700.0,31.01,-1.0,True,True,True,8,700.0,87.500000,2.707020,1.084352,1.622668,12,701.0
22059,33678569,8,"[1.0, 4.0]",2,[30.99],1,3186.0,30.99,1.0,True,True,True,7,3186.0,455.142857,11.536015,8.734460,2.801555,14,3101.0


In [126]:
# look at only orders that were submitted at BBO
order_metrics = order_metrics[order_metrics['Sub_At_BBO']==True]

In [128]:
# calculate mean time to first execution based on queue position
# drop any observations with no execution
first_ex_stats = order_metrics[order_metrics['Any_Ex']==True]
first_ex_stats = first_ex_stats.groupby('Start_Position').agg(Time_To_First_Ex=('First_Ex_Time', 'mean'), Frequency=('Start_Position', 'count')).reset_index()
# drop any queue positions that we have 0 observations for
first_ex_stats = first_ex_stats[first_ex_stats['Frequency']!=0]
print(sum(first_ex_stats['Frequency']))
first_ex_stats.head()

3217


,Start_Position,Time_To_First_Ex,Frequency
0,1,12.785354,299
1,2,14.942040,260
2,3,12.252787,224
3,4,9.636374,165
4,5,15.007652,171


In [129]:
fig = go.Figure(go.Bar(
    x=first_ex_stats['Start_Position'],
    y=first_ex_stats['Frequency'],
    name = 'Frequency'))

# Add time trace
fig.add_trace(go.Scatter(
    x=first_ex_stats['Start_Position'],
    y=first_ex_stats['Time_To_First_Ex'],
    mode='lines',
    name = 'Average Time to First Ex'))

# Update layout
fig.update_layout(
    title='Average Time to First Execution Given Queue Position at Submission',
    xaxis_title='Queue Position At Submission',
    yaxis_title='Frequency',
    yaxis = dict(side='right', showgrid=False),
    yaxis2 = dict(side='left', overlaying='y', title='Average Time to First Execution (Seconds)'))
fig.data[0].yaxis='y'
fig.data[1].yaxis='y2'
fig.update_layout(width=1440, height=600, margin=dict(l=50, r=50, b=75, t=50))
# Show the figure
# fig.show()


Here is the average time to first execution based on the order's position in queue when it is submitted. This graph is genereated by reading 100,000 lines of the message file. Interestingly, it is relativley flat for the first 20 positions. As the observation frequency decreases, we see more noise in the average - in the future we could ammend this by using more data.

!['queue position'](../data/images/queue_position.png)